In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from mindvision.classification.models import resnet50
import mindspore as ms

def output(image_paths):
    str = ""
    for image_path in image_paths:
        image = Image.open(image_path).convert("RGB")
        image = image.resize((224, 224))

        # 归一化处理
        mean = np.array([0.485 * 255, 0.456 * 255, 0.406 * 255])
        std = np.array([0.229 * 255, 0.224 * 255, 0.225 * 255])
        image = np.array(image)
        image = (image - mean) / std
        image = image.astype(np.float32)

        # 图像通道由(h, w, c)转换为(c, h, w)
        image = np.transpose(image, (2, 0, 1))

        # 扩展数据维数为(1, c, h, w)
        image = np.expand_dims(image, axis=0)

        # 定义并加载网络
        net = network
        param_dict = ms.load_checkpoint("./new_checkpoints/best.ckpt")
        ms.load_param_into_net(net, param_dict)
        model = ms.Model(net)

        # 模型预测
        pre = model.predict(ms.Tensor(image))
        result = np.argmax(pre)
        

        class_name = {0: "Bear", 1: "Bird", 2: "Cat", 3: "Cow", 4: "Deer", 5: "Dog", 6: "Dolphin", 7: "Elephant", 8: "Giraffe", 9: "Horse", 10: "Kangaroo", 11: "Lion", 12: "Panda", 13: "Tiger", 14: "Zebra"}
        str += class_name[result] + ","
        
        
    return str


In [ ]:
import mindspore
from mindnlp.core import no_grad
from mindnlp.transformers import AutoModelForCausalLM, AutoTokenizer
from mindspore._c_expression import _framework_profiler_step_start
from mindspore._c_expression import _framework_profiler_step_end
# 加载tokenizer文件

file_paths = []

tokenizer = AutoTokenizer.from_pretrained("ZhipuAI/glm-4-9b-chat", mirror='modelscope')

str = output(file_paths)

# 在此输入希望和ChatGLM4对话的内容
query = f"你是一个陪伴4到6岁孩子玩耍的智能机器人，现在你正在陪一个小男孩玩讲故事游戏，小朋友向你展示了四只小动物玩具：他们是{str}，请你按照固定的出场顺序，以大森林为背景，用这几只小动物为主角给你陪伴的小朋友讲一个引人入胜的童话故事。注意：1. 每个小动物应该有固定的人设，包括性格特征和背景设定，不可更改2. 故事要有波澜，但是整体应该温馨正能量3. 请你注意表达方式，尽可能用小朋友听得懂的话给孩子口述故事"

inputs = tokenizer.apply_chat_template([{"role": "user", "content": query}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="ms",
                                       return_dict=True
                                       )
print(inputs)
# 加载模型
model = AutoModelForCausalLM.from_pretrained(
    "ZhipuAI/glm-4-9b-chat",
    mirror='modelscope',
    ms_dtype=mindspore.float16,
).eval()

# 设置采样、最大生成长度等配置，开始推理
# _framework_profiler_step_start()
gen_kwargs = {"max_length": 1000, "do_sample": True, "top_k": 1}
with no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
# _framework_profiler_step_end()